In [2]:
import pickle
import faiss
import numpy as np

class ComplaintRetriever:
    def __init__(self, index_path, metadata_path, embed_model):
        self.index = faiss.read_index(index_path)
        with open(metadata_path, "rb") as f:
            self.metadata = pickle.load(f)
        self.embed_model = embed_model

    def retrieve(self, query, k=5):
        query_vec = self.embed_model.encode([query])
        distances, indices = self.index.search(query_vec, k)
        results = [self.metadata[i] for i in indices[0]]
        return results


In [ ]:
from sentence_transformers import SentenceTransformer
from rag.retriever import ComplaintRetriever
from rag.generator import ComplaintGenerator

class RAGPipeline:
    def __init__(self, index_path, metadata_path):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.retriever = ComplaintRetriever(index_path, metadata_path, self.embedder)
        self.generator = ComplaintGenerator()

    def query(self, question):
        chunks = self.retriever.retrieve(question)
        answer = self.generator.generate_answer(chunks, question)
        return answer, chunks